In [384]:
import numpy as np
import matplotlib.pyplot as plt
import astropy

import astropy.io.fits as fits
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u
from astropy.units import Quantity
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord

import astroquery
from astroquery.gaia import Gaia
from astroquery.mast import Observations
from astroquery.sdss import SDSS

import drizzlepac
from drizzlepac import tweakreg

import json, requests

from reproject import reproject_interp

import warnings

import os

import scipy
from scipy.ndimage.interpolation import shift

from astropy.wcs import WCS

In [385]:
import artpop

In [11]:
# change this later so that only user inputted value is "file"
file1 = 'm81_optical.fits'
file2 = 'm81_ir.fits' # for some reason this file doesn't have WCS?

In [323]:
def open_fits(fitspath):
    '''
    Opens a fits file. Determines which extension to use, assuming it is either 1 or 0.
    
    Parameters
    ----------
    fitspath:
        The path of the FITS file to be opened.
        
    Returns
    -------
    image: FITS object
    '''
    hdu = fits.open(fitspath)
    image = hdu[0]
    if type(image.data) != np.ndarray:
        image = hdu[1]
    elif type(image.data) != np.ndarray:
        print('Cannot open FITS file',fitspath,'with extension 1 or 0')
        
    hdu.close()
    return image

In [21]:
def counts_to_flux(fitspath):
    '''
    Turns image counts into flux values. Doesn't return anything; instead just modifies the file
    
    Parameters
    ----------
    fitspath: FITS file
        The fits file that needs to be converted to flux values.
    '''
    # ADD HAS ATTRIBUTE WHEN I TURN INTO A CLASS SO IT ONLY RUNS ONCE
    image = open_fits(fitspath)
    try: 
        photflam=image.header['PHOTFLAM']
    except: 
        photflam=2.901E-19
    try: 
        photplam=image.header['PHOTPLAM']
    except: 
        photplam=16030.43
    data = image.data
    zpab = -2.5*np.log10(photflam)-5*np.log10(photplam)-2.408
    magab = -2.5*np.log10(data)+zpab
    data = magab * 606 * 10**9 / (2.9979 * 10**8)
    image.data = data
    pass
    
counts_to_flux(file1)
counts_to_flux(file2)

In [22]:
# this is new code to be applicable to others
def mastQuery(request, url='https://mast.stsci.edu/api/v0/invoke'):
    '''
    Perform a MAST query for an astronomical object.

    Parameters
    ----------
    request: dictionary 
        The MAST request json object
    url: string 
        The service URL. Default is MAST

    Returns
    -------
    r.text: string
        Returns the coordinates of requested astronomical object.
    '''
    
    # Encoding the request as a json string
    requestString = json.dumps(request)
    r = requests.post(url, data={'request': requestString})
    r.raise_for_status()
    return r.text

In [23]:
# this is new code to be applicable to others
def resolve(name):
    '''
    Get the RA and Dec for an object using the MAST name resolver.
    
    Parameters
    ----------
    name: string
        Name of object

    Returns
    -------
    objRa: float
        RA position
    objDec: float
        Dec position
    '''

    resolverRequest = {'service':'Mast.Name.Lookup',
                       'params':{'input':name,
                                 'format':'json'
                                },
                      }
    resolvedObjectString = mastQuery(resolverRequest)
    resolvedObject = json.loads(resolvedObjectString)
    # The resolver returns a variety of information about the resolved object, 
    # however for our purposes all we need are the RA and Dec
    try:
        objRa = resolvedObject['resolvedCoordinate'][0]['ra']
        objDec = resolvedObject['resolvedCoordinate'][0]['decl']
    except IndexError as e:
        raise ValueError("Unknown object '{}'".format(name))
    return (objRa, objDec)

In [38]:
# align to GAIA
def gaia_align(fitspath,objname,radius = 6.0,minobj=2,sigma=5.0,threshold=5., conv_width=6):
    '''
    Aligns a given FITS file to the GAIA catalog. Doesn't return anything; just modifies the file.
    To be used only if a FITS file doesn't appear to have WCS or the WCS seems very off.
    
    Parameters
    ----------
    fitspath: string
        The location of a specific FITS file.
        
    objname: string
        An astronomical object we are aligning an image towards in the GAIA catalog.
        
    radius: string
        The radius in arcseconds that we will search around the given object.
        
    minobj: integer
        The minimum number of objects that need to be found in order for the FITS file to be aligned with the GAIA catalog.
    
    sigma: float
        The allowed error for the alignment process.
        
    threshold: float
        The minimum threshold for finding matchign objects.
        
    conv_width: int
        The minimum convergence width for finding objects.
    '''
    coord = resolve(objname)
    radius = Quantity(radius, u.arcmin)
    coord = SkyCoord(ra=coord[0], dec=coord[1], unit=(u.deg, u.deg))
    gaia_query = Gaia.query_object_async(coordinate=coord, radius=radius)
    
    reduced_query = gaia_query['ra', 'dec', 'phot_g_mean_mag']
    reduced_query.write('gaia.cat', format='ascii.commented_header')
    

    cw = 3.5  # Set to two times the FWHM of the PSF.
    wcsname = 'Gaia'  # Specify the WCS name for this alignment

    # ALIGNING TO GAIA
    if os.path.isdir('gaia.cat'): #rewrite gaia.cat if necessary
        refcat = None
    refcat = 'gaia.cat'
    
    # I got TweakReg from STSci but I wrote the rest of this function
    tweakreg.TweakReg(fitspath,  # Pass input images
                      updatehdr=True,  # update header with new WCS solution
                      imagefindcfg={'threshold':threshold,'conv_width':conv_width},  # Detection parameters
                                                                     # threshold varies for different data
                      refcat=refcat,  # Use user supplied catalog (Gaia)
                      interactive=False,
                      see2dplot=False,
                      minobj = minobj,
                      shiftfile=True,  # Save out shift file (so we can look at shifts later)      
                      wcsname=wcsname,  # Give our WCS a new name
                      reusename=True,
                      sigma=sigma,
                      ylimit=0.2,
                      fitgeometry='general')  # Use the 6 parameter fit  

In [36]:
gaia_align(file1,'m81')

INFO: Query finished. [astroquery.utils.tap.core]
Setting up logfile :  tweakreg.log
TweakReg Version 1.4.7(18-April-2018) started at: 13:49:30.148 (25/11/2021) 

Version Information
--------------------
Python Version [Clang 10.0.0 ]
3.8.12 (default, Oct 12 2021, 06:23:56) 
numpy Version -> 1.21.2 
astropy Version -> 4.3.1 
stwcs Version -> 1.7.0 

Finding shifts for: 
    m81_optical.fits

===  Source finding for image 'm81_optical.fits':
  #  Source finding for 'm81_optical.fits', EXT=('SCI', 1) started at: 13:49:30.275 (25/11/2021)


     Found 1751 objects.
===  FINAL number of objects in image 'm81_optical.fits': 1751


Performing alignment in the projection plane defined by the WCS
derived from 'm81_optical.fits'


Performing fit for: m81_optical.fits

Matching sources from 'm81_optical.fits' with sources from reference catalog 'gaia.cat'
Computing initial guess for X and Y shifts...
Found initial X and Y shifts of 1, 1.217 with significance of 17.16 and 23 matches
##############################################################################
#                                                                            #
# WARNING:                                                                   #
# Not enough matches (< 2) found for input image: m81_optical.fits           #
#                                                                            #
##############################################################################

Unable to match the following images:
-------------------------------------
m81_op

In [37]:
gaia_align(file2,'m81')

INFO: Query finished. [astroquery.utils.tap.core]
Setting up logfile :  tweakreg.log
TweakReg Version 1.4.7(18-April-2018) started at: 13:49:35.384 (25/11/2021) 

Version Information
--------------------
Python Version [Clang 10.0.0 ]
3.8.12 (default, Oct 12 2021, 06:23:56) 
numpy Version -> 1.21.2 
astropy Version -> 4.3.1 
stwcs Version -> 1.7.0 

Finding shifts for: 
    m81_ir.fits

===  Source finding for image 'm81_ir.fits':
  #  Source finding for 'm81_ir.fits', EXT=('SCI', 1) started at: 13:49:35.464 (25/11/2021)
     Found 350 objects.
===  FINAL number of objects in image 'm81_ir.fits': 350

Trailer file written to:  tweakreg.log


TypeError: unsupported operand type(s) for /: 'NoneType' and 'float'

TypeError: unsupported operand type(s) for /: 'NoneType' and 'float'

In [39]:
# align files to each other
def filealign(fits_comparison, fits_shift, minobj=2):
    '''
    Aligns a given FITS file with another fits file. Doesn't return anything; just modifies the file.
    Preferably the chi2_images function should be used over this one.
    
    Parameters
    ----------
    fits_comparison: string
        The location of the FITS file that will be compared against.
        
    fits_shift: string
        The location of the FITS file that will be shifted.
        
    minobj: integer
        The minimum number of objects that need to be found in order for the FITS file to be aligned with the GAIA catalog.
    '''
    tweakreg.TweakReg(fits_comparison,
                      enforce_user_order=False,
                      imagefindcfg={'threshold': 10, 'conv_width': 3.5, 'dqbits': ~4096},
                      minobj = minobj,
                      refimage=fits_shift, 
                      refimagefindcfg={'threshold': 10, 'conv_width': 2.5},
                      shiftfile=True,
                      outshifts='shift657_flc.txt',
                      searchrad=5.0,
                      ylimit=0.6,
                      updatehdr=True,
                      #updatewcs=True,
                      wcsname='UVIS_FLC',
                      reusename=True,
                      interactive=False)  

In [40]:
filealign(file1,file2)

Setting up logfile :  tweakreg.log
TweakReg Version 1.4.7(18-April-2018) started at: 13:53:10.335 (25/11/2021) 

Version Information
--------------------
Python Version [Clang 10.0.0 ]
3.8.12 (default, Oct 12 2021, 06:23:56) 
numpy Version -> 1.21.2 
astropy Version -> 4.3.1 
stwcs Version -> 1.7.0 

Finding shifts for: 
    m81_optical.fits

===  Source finding for image 'm81_optical.fits':


     #########################################################################
     #                                                                       #
     # WARNING: User requested to use DQ mask for source finding, but DQ     #
     # data are not available.                                               #
     # DQ mask WILL NOT be used for source finding.                          #
     #                                                                       #
     #########################################################################


  #  Source finding for 'm81_optical.fits', EXT=('SCI', 1) started at: 13:53:10.481 (25/11/2021)
     Found 1634 objects.
===  FINAL number of objects in image 'm81_optical.fits': 1634

===  Source finding for image 'm81_ir.fits':
  #  Source finding for 'm81_ir.fits', EXT=('SCI', 1) started at: 13:53:11.925 (25/11/2021)
     Found 115 objects.
===  FINAL number of objects in image 'm81_ir.fits': 115

Trailer file written to:  tweakreg.log


TypeError: unsupported operand type(s) for /: 'NoneType' and 'float'

TypeError: unsupported operand type(s) for /: 'NoneType' and 'float'

In [271]:
def reproject(largefitspath,smallfitspath,largeext,smallext,reproject_name):
    '''
    Reproject two FITS images so that the larger image is on the same pixel scale as the smaller image.
    
    Parameters
    ----------
    largefitspath: string
        The location of a specific FITS file. This is the larger image.
        
    smallfitspath: string
        The location of a specific FITS file.
        
    reproject_name: string
        The name of the reprojected FITS file.
        
    largeext: string
        The extension of the first FITS file.
        
    smallext: string
        The extensino of the second FITS file.
    
    Returns
    -------
    reproject_path: string
        Returns the path location of the reprojected FITS file.
    '''
    largehdu = fits.open(largefitspath)
    largeimage=largehdu[largeext] # can't use my fxn because i need the 'sci' and 'wht' keywords
    largehdu.close()
    smallhdu = fits.open(smallfitspath)[smallext]
    smallimage=smallhdu[smallext]
    smallhdu.close()

    array, footprint = reproject_interp(smallimage, largeimage.header)

    # this all plots the reprojection
#     ax1 = plt.subplot(1,2,1, projection=WCS(largeimage.header))
#     ax1.imshow(array, vmin=0, vmax=15,cmap='hot')
#     ax1.coords['ra'].set_axislabel('Right Ascension')
#     ax1.coords['dec'].set_axislabel('Declination')

#     ax2 = plt.subplot(1,2,2, projection=WCS(largeimage.header))
#     ax2.imshow(footprint, vmin=0, vmax=15)
#     ax2.coords['ra'].set_axislabel('Right Ascension')
#     ax2.coords['dec'].set_axislabel('Declination')
#     ax2.coords['dec'].set_axislabel_position('r')
#     ax2.coords['dec'].set_ticklabel_position('r')

    reproject_path = reproject_name + '.fits'

    fits.writeto(reproject_path, array, largeimage.header, overwrite=True)
    
    return reproject_path

In [272]:
fits.open(file1).info()

Filename: m81_optical.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     778   ()      
  1  SCI           1 ImageHDU       123   (5725, 5735)   float32   
  2  WHT           1 ImageHDU        44   (5725, 5735)   float32   
  3  CTX           1 ImageHDU        37   (5725, 5735)   int32   
  4  HDRTAB        1 BinTableHDU    638   2R x 314C   [9A, 3A, K, D, D, D, D, D, D, D, D, D, D, D, D, D, K, 4A, 4A, 1A, 4A, 4A, D, D, D, D, 3A, D, D, D, D, D, D, D, D, D, D, D, D, K, 8A, 23A, D, D, D, D, K, K, K, 8A, K, 23A, 9A, 20A, K, 4A, K, K, K, K, K, K, 23A, D, D, D, D, K, K, 3A, 3A, 4A, 4A, L, D, D, D, 3A, 1A, K, D, D, D, 13A, 3A, 4A, 4A, 12A, 12A, 23A, 8A, 23A, 10A, 10A, D, D, 3A, 3A, 23A, 4A, 8A, 7A, 23A, D, K, D, 6A, 9A, 8A, D, D, L, 4A, 43A, 3A, K, 7A, 5A, 3A, D, 13A, 8A, 4A, 3A, L, K, L, K, L, K, K, D, D, D, D, D, D, 3A, 1A, D, 23A, D, D, D, 3A, 23A, L, 1A, 3A, 1A, D, 3A, 6A, K, D, D, D, D, D, D, D, D, D, D, 23A, D, D, D, D, 3A, D, D, D, 1

In [273]:
def reproject_science(file1,file2,reproject_name):
    reproject_path = reproject(file1,file2,'SCI','SCI',reproject_name)
    return reproject_path

def reproject_error(file1,file2,reproject_name):
    if 'WHT' in np.asarray(fits.open(file1).info()):
        # adjust weights to errors
        reproject_err_path = reproject(file1,file2,'WHT','SCI',reproject_name)
        image = fits.open(reproject_name)[0]
        data = 1/image.data
        image.data = data
        return reproject_err_path
        
    if 'ERR' in np.asarray(fits.open(file1).info()):
        reproject_err_path = reproject(file1,file2,'ERR','SCI',reproject_name)
        return reproject_err_path

    else:
        print('No error file present')
        return None

In [363]:
# FINALLY: CHI2 ACTION!!!

## get errors
## add y direction

def shift_images(fits_comparison,fits_shift,pixelrange):
    '''
    Aligns a given FITS file with another fits file by subtracting the two images and then minimizing the added flux. 
    Checks along a range of different pixel values for the total chi2 and then 
    
    Parameters
    ----------
    fits_comparison: string
        The location of the FITS file that will be compared against.
        
    fits_shift: string
        The location of the FITS file that will be shifted.
        
    pixelrange: integer
        The range of pixels to adjust the search over. Ranges from negative of that value to positive of that value.
        
    Returns
    -------
    image_shift: ndarray
        The first FITS file as an array shifted by the amount necessary to minimize the subtraction with the second FITS file.
        
    x_min: float
        The amount to shift the image by in the x direction.
        
    y_min: float
        The amount to shift the image by in the y direction.
    '''
    reproject_path = reproject_science(fits_shift,fits_comparison,fitspath1+'_reproject')
    reproject_err_path = reproject_error(fits_shift,fits_comparison,fitspath1+'_reproject_err')
    
    # figure out the extensions
    # can't use my function since can't close files till later
    hdu_comparison = fits.open(reproject_path)
    image_comparison = hdu_comparison[0]
    if type(image_comparison.data) != np.ndarray:
        image_comparison = hdu_comparison[1]
        
    hdu_shift = fits.open(fits_shift)
    image_shift = hdu_shift[0]
    if type(image_shift.data) != np.ndarray:
        image_shift = hdu_shift[1]
    
    # if error path exists, check extensions 
    if reproject_err_path is not None:
        hdu_err = fits.open(reproject_err_path)
        err = hdu_err[0]
        if type(err.data) != np.ndarray:
            err = hdu_err[1]      
    else: # if no error path exists just set err to None
        err = None
    
    pixel_grid  = np.arange(-pixelrange,pixelrange,1)

    x_arr,y_arr,chi2 = [],[],[]
    chi2_min = 1e20
    x_min, y_min = 0,0

    image = np.zeros(shape=(len(pixel_grid),len(pixel_grid)))
    chi2 = []
    for i,x in enumerate(pixel_grid):
        for j,y in enumerate(pixel_grid):
            
            
            # MOVE THE IMAGE OVER
            image_xshift = np.roll(image_shift.data,pixel_grid[i],axis=1) # shifting image in x range
            image_xshift[:,pixel_grid[i]] = np.mean(image2.data) # fills in edges with the mean of the data instead
            image_xyshift = np.roll(image_xshift,pixel_grid[j],axis=0)
            image_xyshift[:,pixel_grid[j]] = np.mean(image2_xshift.data)
            
            # CALCULATE CHI2
            subtraction = np.abs(image_comparison.data - image_xyshift) # need absolute value otherwise adding it all up doesn't make sense as negatives cancel each other
            flux_sum = np.sum(subtraction)

            if err is not None:
                c = (flux_sum-err)**2
                
            else:
                c = flux_sum**2

            chi2.append(c)
            x_arr.append(x)
            y_arr.append(y)
            image[i,j] = c
            if c < chi2_min:
                chi2_min = c
                x_min    = x
                y_min    = y

    chi2 = np.array(chi2)
    x_arr = np.array(x_arr)
    y_arr = np.array(y_arr)
    
    final_image_xshift = np.roll(image_shift.data,x_min,axis=1) # fills in edges with zeroes; shifting image in both x and y range
    final_image_xyshift = np.roll(final_image_xshift,y_min,axis=0)
    
    hdu_comparison.close()
    hdu_shift.close()
    if reproject_err_path is not None:
        hdu_err.close()
    
    return final_image_xyshift,x_min,y_min

In [356]:
final_image_xyshift,x_min,y_min=shift_images(file1,file2,30)

Filename: m81_ir.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     323   ()      
  1  SCI           1 ImageHDU       143   (256, 256)   float32   
  2  ERR           1 ImageHDU        71   (256, 256)   float32   
  3  DQ            1 ImageHDU        71   (256, 256)   int16   
  4  SAMP          1 ImageHDU        71   (256, 256)   int16   
  5  TIME          1 ImageHDU        71   (256, 256)   float32   
Filename: m81_ir.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     323   ()      
  1  SCI           1 ImageHDU       143   (256, 256)   float32   
  2  ERR           1 ImageHDU        71   (256, 256)   float32   
  3  DQ            1 ImageHDU        71   (256, 256)   int16   
  4  SAMP          1 ImageHDU        71   (256, 256)   int16   
  5  TIME          1 ImageHDU        71   (256, 256)   float32   
No error file present


In [381]:
def shift_save(fits_comparison,fits_shift,pixelrange,outputfilename,directory=os.getcwd()):
    '''
    Shifts a fits file so that when subtracted from another file, its chi2 is minimized. Saves the shifted array. If update is True, it updates the old file to the new data.
    
    Parameters
    ----------
    fits_comparison: string
        The location of the FITS file that will be shifted.
        
    fits_shift: string
        The location of the FITS file that will be compared against.
        
    pixelrange: integer
        The range of pixels to adjust the search over. Ranges from negative of that value to positive of that value.
        
    outputfilename: string
        What to name the new FITS file.
        
    directory: string
        What directory to save the new FITS file to. Default is the current working directory.
    '''
    output_arr,x_min,y_min = shift_images(fits_comparison,fits_shift,pixelrange)
    hdu = fits.PrimaryHDU(data=output_arr)
    hdu.writeto(directory+'/'+outputfilename)
    
    output = fits.open(outputfilename)
    
    image_shift = fits.open(fits_shift)[0]
    if type(image_shift.data) != np.ndarray:
        image_shift = fits.open(fits_shift)[1]

    w = WCS(image_shift.header)
    
    # change the header to the wcs from the other file
    output[0].header.update(w.to_header())

    output.close()
    pass

In [383]:
shift_save(file1,file2,30,'shif3.fits')

Filename: m81_ir.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     323   ()      
  1  SCI           1 ImageHDU       143   (256, 256)   float32   
  2  ERR           1 ImageHDU        71   (256, 256)   float32   
  3  DQ            1 ImageHDU        71   (256, 256)   int16   
  4  SAMP          1 ImageHDU        71   (256, 256)   int16   
  5  TIME          1 ImageHDU        71   (256, 256)   float32   
Filename: m81_ir.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     323   ()      
  1  SCI           1 ImageHDU       143   (256, 256)   float32   
  2  ERR           1 ImageHDU        71   (256, 256)   float32   
  3  DQ            1 ImageHDU        71   (256, 256)   int16   
  4  SAMP          1 ImageHDU        71   (256, 256)   int16   
  5  TIME          1 ImageHDU        71   (256, 256)   float32   
No error file present


In [354]:
fitspath1=file1
fitspath2=file2
reproject_path = reproject_science(fitspath2,fitspath1,fitspath1+'_reproject')
reproject_err_path = reproject_error(fitspath2,fitspath1,fitspath1+'_reproject_err')

# first try ext = 1
image1 = fits.open(reproject_path)[0]
if image1 == None:
    image1 = fits.open(reproject_path)[1]

image2 = fits.open(fitspath2)[1]
if reproject_err_path is not None:
    err = fits.open(reproject_err_path)
else:
    err = None

pixel_grid  = np.arange(-pixelrange,pixelrange,1)

x_arr,y_arr,chi2 = [],[],[]
chi2_min = 1e20
x_min, y_min = 0,0

image = np.zeros(shape=(len(pixel_grid),len(pixel_grid)))
chi2 = []
for i,x in enumerate(pixel_grid):
    for j,y in enumerate(pixel_grid):


        # MOVE THE IMAGE OVER
        image2_xshift = np.roll(image2.data,pixel_grid[i],axis=1) # fills in edges with zeroes; shifting image in both x and y range
        image2_adjusted = np.roll(image2_xshift,pixel_grid[j],axis=0)

        # CALCULATE CHI2
        subtraction = np.abs(image1.data - image2_adjusted) # need absolute value otherwise adding it all up doesn't make sense as negatives cancel each other
        flux_sum = np.sum(subtraction)

        if err is not None:
            c = (flux_sum-err)**2

        else:
            c = flux_sum**2

        chi2.append(c)
        x_arr.append(x)
        y_arr.append(y)
        image[i,j] = c
        if c < chi2_min:
            chi2_min = c
            x_min    = x
            y_min    = y

chi2 = np.array(chi2)
x_arr = np.array(x_arr)
y_arr = np.array(y_arr)

final_image2_xshift = np.roll(image2.data,x_min,axis=1) # fills in edges with zeroes; shifting image in both x and y range
final_image2_adjusted = np.roll(image2_xshift,y_min,axis=0)

Filename: m81_ir.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     323   ()      
  1  SCI           1 ImageHDU       143   (256, 256)   float32   
  2  ERR           1 ImageHDU        71   (256, 256)   float32   
  3  DQ            1 ImageHDU        71   (256, 256)   int16   
  4  SAMP          1 ImageHDU        71   (256, 256)   int16   
  5  TIME          1 ImageHDU        71   (256, 256)   float32   
Filename: m81_ir.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     323   ()      
  1  SCI           1 ImageHDU       143   (256, 256)   float32   
  2  ERR           1 ImageHDU        71   (256, 256)   float32   
  3  DQ            1 ImageHDU        71   (256, 256)   int16   
  4  SAMP          1 ImageHDU        71   (256, 256)   int16   
  5  TIME          1 ImageHDU        71   (256, 256)   float32   
No error file present


# bugs:
#### shifting
- can't do non full step pixels bc then i would use scipy.ndimage.shift and that interpolates half a value between zero and the edge
    - could potentially be fixed by using the mean instead of zero at the edges but i'd need to do the math to figure out whether that makes sense...
    - not sure whether it's worthwhile for now using np.roll
        - image2_adjusted = shift(image2.data,[pixelx_grid[i],pixely_grid[j]],mode = 'constant',cval=0.0) # fills in edges with zeroes; shifting image in both x and y range
        - hypothetically could make pixel grids be non intenger steps
        
#### TweakReg
- doesn't work if no WCS: how to raise an exception?


#### Updating shifted file
- should I do it or just new file forget it


#### CLOSING FILES!

#### when do you use pass?